# Exercício: Análise de Sentimento de Avaliações de Clientes
* **Descrição:** Classificar reviews Olist em positivas/negativas para medir satisfação.
Dataset: Brazilian E-Commerce Public Dataset (Kaggle) — tabela olist_order_reviews_dataset.csv
* **Dataset:** Dataset público de e-commerce publicado pela empresa __olist.com__
  * **Download:** https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

## Passo a passo

1. Ler CSV; criar rótulo: positiva se review_score≥4, negativa se ≤2.
2. Separar treino/teste 80 / 20.
3. TfidfVectorizer(max_features=10 000, ngram_range=(1,2)) sobre review_comment_message.
4. Treinar LogisticRegression(max_iter=1000).
5. Avaliar precisão, recall, F1; inspecionar 5 erros mais graves.
6. Gerar nuvem de palavras por classe para insights.

## Download do Dataset


*   Do AWS: https://genaiacademy.s3.eu-west-3.amazonaws.com/datasets/olist/olist_order_reviews_dataset.csv
*   Do Kaggle: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
* Arquivo: olist_order_reviews_dataset.csv


In [ ]:
# Opcional: Se quizer fazer o download do arquivo direto do meu Google Drive

# NOTA: Em ambiente do Google colab o gdown já funciona diretamente
# caso necessário pode instalá-lo via pip, descomentanto a linha abaixo:
# !pip install gdown

!wget  https://genaiacademy.s3.eu-west-3.amazonaws.com/datasets/olist/olist_order_reviews_dataset.csv

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# 1. Carregar dados
df = pd.read_csv('olist_order_reviews_dataset.csv')
df = df[df['review_score'].isin([1,2,4,5])].dropna(subset=['review_comment_message'])

# 2. Definir labels
df['label'] = (df['review_score']>=4).astype(int)  # 1=positivo

In [ ]:
# 3. Fazer o split entre dados de treinamento e os de teste
X_train, X_test, y_train, y_test = train_test_split(
    df['review_comment_message'], df['label'], test_size=0.2, random_state=42)


In [ ]:
# (Opcional) - Análise exploratória de dados, mostrando
# nuvem de palavras positivas
wc = WordCloud(width=600, height=400).generate(' '.join(X_train[y_train==1]))
plt.imshow(wc); plt.axis('off')

In [ ]:
# nuvem de palavras negativas
wc2 = WordCloud(width=600, height=400).generate(' '.join(X_train[y_train!=1]))
plt.imshow(wc2); plt.axis('off')

In [ ]:
# 4. Vectorizar
tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1,2), stop_words=None)
Xtr = tfidf.fit_transform(X_train)
Xte = tfidf.transform(X_test)

# 5. Treinar
clf = LogisticRegression(max_iter=1000, n_jobs=-1)
clf.fit(Xtr, y_train)
print(classification_report(y_test, clf.predict(Xte)))

## Testar manualmente o modelo treinado
* Já temos um classificador pronto
* Agora seria interessante aproveitar o modelo e testá-lo com frases arbitrárias fornecidas por nós
* **Ex: Entradas teste manual**
  * nunca mais compro
  * muito obrigado
  * não recebi o produto
  * vocês são péssimos
  * gostei demais

In [ ]:
# 6. [Extra] Usar o modelo treinado para prever frases de input pelo user
user_review = input("Qual o seu feedback da compra?: ")
sentimmetos = ["Negativo 🚫", "Positivo ✅"]
previsto = clf.predict(tfidf.transform([user_review]))
print("Sentimento previsto:", sentimmetos[previsto[0]])